In [23]:
# Import the dependencies.
# Import the datetime module from the datetime library.
from datetime import datetime
import pandas as pd
import numpy as np
import requests
from citipy import citipy
import json
import subprocess
import psycopg2

In [24]:
# Import the API key.
from config import client_id, client_secret, redirect_url, base_url, oauth_url

In [25]:
df =  pd.read_csv("//SERVER/Home$/jbell/Convert.csv")

In [28]:
df.head()

,Columns A & B should be linked to CostCode Table,Unnamed: 1,Exclude Comments,Current Budget,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Internal Estimate,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Qualifications,Unnamed: 23,Unnamed: 24,Specs,Assumptions,Alternates,Alt Included,Unnamed: 29,"Bidder's & Amounts - Unfortunately without dates from the Scope Sheets, this is not useful to send to the DB. Something for future would be to pull the bidders, amounts, and dates from the individual scope sheets - ie: 01-06-120 - Surveying",Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Other Project Costs (per NRSF & Unit),Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Other Project Differences (per NRSF & Unit),Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,NRSF Cost Tracking,Unnamed: 58,Unnamed: 59,Unnamed: 60,Other Project Costs ($ Amount),Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Other Project Differences ($ Amount),Unnamed: 69,Unnamed: 70,Unnamed: 71,$ Cost Tracking,Unnamed: 73,Unnamed: 74,Unnamed: 75,CSI Division Mapping,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unit/NRSF/GCs Separation,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126
0,Project Name:,Broadmoor Block F,-,"48,473,675",NaN,NaN,Spread Variance %:,NaN,NaN,"45,716,113",#VALUE!,205.90,194.18,#VALUE!,"178,212","168,074",#VALUE!,"134,276","126,637","(48,473,675)","2,757,562","(45,716,113)",NaN,NaN,NaN,Trade Filter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bid Analysis,195.80,"151,703",213.58,"171,077",189.18,"141,514",NaN,199.52,"154,765",NaN,$/SF & Unit,195.80,"151,703",213.58,"171,077",189.18,"141,514",$/SF & Unit Differences,Highlight:,$2.00,/NRSF lower,$/SF & Unit Highlight Settings,"47,634,851","50,125,619","45,425,875",NaN,"47,728,781.63",NaN,$ Amounts,"47,634,851","50,125,619","45,425,875",$ Amount Differences,Highlight:,"$250,000",lower,$ Amount Highlight Settings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Developer:,Brandywine,-,"53,321,043",NaN,NaN,NaN,NaN,NaN,"50,287,724",#VALUE!,226.49,213.60,#VALUE!,"196,033","184,881",#VALUE!,"147,704","139,301","(53,321,043)","3,033,319","(50,287,724)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,215.53,"166,994",223.81,"179,269",196.43,"146,940",NaN,211.92,"164,401",NaN,NaN,215.53,"166,994",223.81,"179,269",196.43,"146,940",NaN,Highlight:,$2.00,/NRSF higher,NaN,"52,436,066","52,525,879","47,167,852",NaN,"50,709,932.30",NaN,NaN,"52,436,066","52,525,879","47,167,852",NaN,Highlight:,"$250,000",higher,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Current Plans:,8/27/21 GFF Arch Permit Set,NaN,Permit Set,NaN,NaN,NaN,NaN,NaN,Permit Set,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Filter bids older than this date:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,775,Avg. SF/Unit,801,Avg. SF/Unit,748,Avg. SF/Unit,NaN,774,Avg. SF/Unit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [27]:
df.columns

Index(['Columns A & B should be linked to CostCode Table', 'Unnamed: 1',
       ' Exclude Comments ', ' Current Budget ', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', ' Internal Estimate ',
       ...
       'Unnamed: 117', 'Unnamed: 118', 'Unnamed: 119', 'Unnamed: 120',
       'Unnamed: 121', 'Unnamed: 122', 'Unnamed: 123', 'Unnamed: 124',
       'Unnamed: 125', 'Unnamed: 126'],
      dtype='object', length=127)

In [22]:
display([df.iloc[20]])

[18
 Cost Code                                                 NaN
 Description                                      Assistant #3
 NaN                                                       NaN
  Budget                                                   NaN
 Quantity                                                0.75 
 Unit of Measure                                           EA 
 $/Unit                                             80,000.00 
  Modifier                                               1.14 
 Sub-totals                                           147,767 
 Estimate                                                  NaN
  /NRSF                                                    NaN
  Budget/NRSF                                              NaN
  Estimate/NRSF                                            NaN
  /Unit                                                    NaN
  Budget/Unit                                              NaN
  Estimate/Unit                                    

In [8]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [9]:
df.isna().sum()

Columns A & B should be linked to CostCode Table                                                                                                                                                                                                    749
Unnamed: 1                                                                                                                                                                                                                                          206
 Exclude Comments                                                                                                                                                                                                                                   722
 Current Budget                                                                                                                                                                                                                                     706
Unnamed:

In [10]:
df["Columns A & B should be linked to CostCode Table"].isna().sum()

749

In [11]:
df.columns = df.iloc[18]

In [12]:
df.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19], axis=0, inplace= True)

In [13]:
df_cut = df.loc[20:66]

In [21]:
df_cut.head(3)

18,Cost Code,Description,NaN,Budget,Quantity,Unit of Measure,$/Unit,Modifier,Sub-totals,Estimate,/NRSF,Budget/NRSF,Estimate/NRSF,/Unit,Budget/Unit,Estimate/Unit,/Bed,Budget/Bed,Estimate/Bed,(OVER)/UNDER,(OVER)/UNDER,(OVER)/UNDER,Type,$/NRSF,$/Unit,$/NRSF,$/Unit,$/NRSF,$/Unit,$/NRSF,$/Unit,(10.10),"(26,509)",7.68,"(7,135)",(16.72),"(36,698)","Highlight by NRSF, Unit, or Both","(243,289 / 314)\n$ Amount","(234,694 / 293)\n$ Amount","(240,125 / 321)\n$ Amount",$ Amount,$ Amount,$ Amount,$ Amount,Division,Trades
20,NaN,GENERAL CONDITIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GENERAL CONDITIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,01-01-100,Administrative Overhead,-,"76,448",NaN,NaN,Annual Salary/Total Bonus,Burden+Inflation,NaN,"76,448",#VALUE!,0.32,0.32,#VALUE!,281,281,#VALUE!,212,212,"(76,448)",-,"(76,448)",Est,0.46,357,0.36,290,0.43,324,0.42,324,0.14,76,0.04,9,0.11,43,Both,"112,000","85,000","104,000","100,333","35,552","8,552","27,552",1,General Conditions
22,NaN,CA Salary,NaN,NaN,0.33,EA,"75,000.00",1.29,"68,998",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,General Conditions


In [15]:
df_cut = df_cut.dropna(how="all", axis=1)

In [16]:
df_cut.columns

Index([                       'Cost Code',                      'Description',
                                      nan,                         ' Budget ',
                               'Quantity',                  'Unit of Measure',
                                 '$/Unit',                       ' Modifier ',
                             'Sub-totals',                         'Estimate',
                                ' /NRSF ',                    ' Budget/NRSF ',
                        ' Estimate/NRSF ',                          ' /Unit ',
                          ' Budget/Unit ',                  ' Estimate/Unit ',
                                 ' /Bed ',                     ' Budget/Bed ',
                         ' Estimate/Bed ',                     '(OVER)/UNDER',
                           '(OVER)/UNDER',                     '(OVER)/UNDER',
                                   'Type',                           '$/NRSF',
                                 '$/Unit',          

In [17]:
df_final = df_cut.fillna(0)

In [18]:
df_final

18,Cost Code,Description,NaN,Budget,Quantity,Unit of Measure,$/Unit,Modifier,Sub-totals,Estimate,/NRSF,Budget/NRSF,Estimate/NRSF,/Unit,Budget/Unit,Estimate/Unit,/Bed,Budget/Bed,Estimate/Bed,(OVER)/UNDER,(OVER)/UNDER,(OVER)/UNDER,Type,$/NRSF,$/Unit,$/NRSF,$/Unit,$/NRSF,$/Unit,$/NRSF,$/Unit,(10.10),"(26,509)",7.68,"(7,135)",(16.72),"(36,698)","Highlight by NRSF, Unit, or Both","(243,289 / 314)\n$ Amount","(234,694 / 293)\n$ Amount","(240,125 / 321)\n$ Amount",$ Amount,$ Amount,$ Amount,$ Amount,Division,Trades
20,0,GENERAL CONDITIONS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,GENERAL CONDITIONS,0,0,0,0,0,0,0,0,0
21,01-01-100,Administrative Overhead,-,"76,448",0,0,Annual Salary/Total Bonus,Burden+Inflation,0,"76,448",#VALUE!,0.32,0.32,#VALUE!,281,281,#VALUE!,212,212,"(76,448)",-,"(76,448)",Est,0.46,357,0.36,290,0.43,324,0.42,324,0.14,76,0.04,9,0.11,43,Both,"112,000","85,000","104,000","100,333","35,552","8,552","27,552",1,General Conditions
22,0,CA Salary,0,0,0.33,EA,"75,000.00",1.29,"68,998",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,General Conditions
23,0,Bonus per Employee,0,0,0.33,EA,"15,000.00",0,"4,950",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,General Conditions
24,0,Payroll Processing,0,0,1.00,LS,"2,500.00",0,"2,500",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,General Conditions
25,01-01-110,Project Manager Payroll,-,"374,533",0,0,Annual Salary/Total Bonus,Burden+Inflation,0,"374,533",#VALUE!,1.59,1.59,#VALUE!,"1,377","1,377",#VALUE!,"1,037","1,037","(374,533)",-,"(374,533)",Est,1.20,932,1.51,"1,209",0.80,600,1.17,914,(0.39),(445),(0.08),(168),(0.79),(777),Both,"292,500","354,330","192,500","279,777","(82,033)","(20,203)","(182,033)",1,General Conditions
26,0,Annual Salary,0,0,0.75,EA,"160,000.00",1.29,"334,533",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,General Conditions
27,0,Bonus per Employee,0,0,1.00,EA,"40,000.00",0,"40,000",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,General Conditions
28,01-01-112,Project Coordinator Payroll,-,"114,980",0,0,Annual Salary/Total Bonus,Burden+Inflation,0,"114,980",#VALUE!,0.49,0.49,#VALUE!,423,423,#VALUE!,319,319,"(114,980)",-,"(114,980)",Est,0.24,184,0.32,256,0.22,162,0.26,201,(0.25),(239),(0.17),(167),(0.27),(261),Both,"57,750","75,000","52,000","61,583","(57,230)","(39,980)","(62,980)",1,General Conditions
29,0,Annual Salary,0,0,0.33,EA,"80,000.00",1.29,"73,597",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,General Conditions


In [19]:
df_final.loc[df_final.columns == 'Cost Code']

18,Cost Code,Description,NaN,Budget,Quantity,Unit of Measure,$/Unit,Modifier,Sub-totals,Estimate,/NRSF,Budget/NRSF,Estimate/NRSF,/Unit,Budget/Unit,Estimate/Unit,/Bed,Budget/Bed,Estimate/Bed,(OVER)/UNDER,(OVER)/UNDER,(OVER)/UNDER,Type,$/NRSF,$/Unit,$/NRSF,$/Unit,$/NRSF,$/Unit,$/NRSF,$/Unit,(10.10),"(26,509)",7.68,"(7,135)",(16.72),"(36,698)","Highlight by NRSF, Unit, or Both","(243,289 / 314)\n$ Amount","(234,694 / 293)\n$ Amount","(240,125 / 321)\n$ Amount",$ Amount,$ Amount,$ Amount,$ Amount,Division,Trades
20,0,GENERAL CONDITIONS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,GENERAL CONDITIONS,0,0,0,0,0,0,0,0,0


In [20]:
df_here = df_here.columns.fillna(0)

NameError: name 'df_here' is not defined

In [ ]:
df_final = df_final.loc[df_final.columns == df_here]

In [ ]:
df_final.columns

Index([                       'Cost Code',                      'Description',
                                      nan,                         ' Budget ',
                               'Quantity',                  'Unit of Measure',
                                 '$/Unit',                       ' Modifier ',
                             'Sub-totals',                         'Estimate',
                                ' /NRSF ',                    ' Budget/NRSF ',
                        ' Estimate/NRSF ',                          ' /Unit ',
                          ' Budget/Unit ',                  ' Estimate/Unit ',
                                 ' /Bed ',                     ' Budget/Bed ',
                         ' Estimate/Bed ',                     '(OVER)/UNDER',
                           '(OVER)/UNDER',                     '(OVER)/UNDER',
                                   'Type',                           '$/NRSF',
                                 '$/Unit',          